In [74]:
import pandas as pd
import numpy as np
import cv2
from sklearn.model_selection import train_test_split
import sklearn

# Load the dataset
data_path = '/home/mitnik/Documents/MNA/autonomous_driving/proyecto_final/image_bank/'
driving_log = pd.read_csv('/home/mitnik/Documents/MNA/autonomous_driving/proyecto_final/angle.csv')

# Split the data into training and validation sets
train_samples, validation_samples = train_test_split(driving_log, test_size=0.2)

def preprocess_image(image):
    image = cv2.cvtColor(image, cv2.COLOR_BGR2RGB)
    image = image / 255.0  # Normalize the image
    return image

def generator(samples, batch_size=32):
    num_samples = len(samples)
    while 1:  # Loop forever so the generator never terminates
        sklearn.utils.shuffle(samples)
        for offset in range(0, num_samples, batch_size):
            batch_samples = samples.iloc[offset:offset+batch_size]

            images = []
            angles = []
            for _, batch_sample in batch_samples.iterrows():
                # Load and preprocess center image
                print(batch_sample)
                center_image_path = data_path + batch_sample[0]
                print(center_image_path)
                center_image = cv2.imread(center_image_path)
                center_image = preprocess_image(center_image)
                center_angle = float(batch_sample[1])

                images.append(center_image)
                angles.append(center_angle)

            X_train = np.array(images)
            y_train = np.array(angles)
            yield sklearn.utils.shuffle(X_train, y_train)

train_generator = generator(train_samples)
validation_generator = generator(validation_samples)


In [75]:
import tensorflow as tf
from tensorflow.keras.models import Sequential
from tensorflow.keras.layers import Conv2D, Flatten, Dense, Lambda, Dropout

model = Sequential([
    Lambda(lambda x: x, input_shape=(128, 64, 3)),
    Conv2D(24, (5, 5), strides=(2, 2), activation='relu'),
    Conv2D(36, (5, 5), strides=(2, 2), activation='relu'),
    Conv2D(48, (5, 5), strides=(2, 2), activation='relu'),
    Conv2D(64, (3, 3), activation='relu'),
    Conv2D(64, (3, 3), activation='relu'),
    Flatten(),
    Dense(100, activation='relu'),
    Dropout(0.5),
    Dense(50, activation='relu'),
    Dropout(0.5),
    Dense(10, activation='relu'),
    Dense(1)
])

model.compile(loss='mse', optimizer='adam')

/home/mitnik/.local/lib/python3.11/site-packages/keras/src/layers/core/lambda_layer.py:67: UserWarning: Do not pass an `input_shape`/`input_dim` argument to a layer. When using Sequential models, prefer using an `Input(shape)` object as the first layer in the model instead.
  super().__init__(**kwargs)


In [76]:
history = model.fit(train_generator,
                    steps_per_epoch=len(train_samples) // 32,
                    validation_data=validation_generator,
                    validation_steps=len(validation_samples) // 32,
                    epochs=5)

/tmp/ipykernel_203347/3652791522.py:33: FutureWarning: Series.__getitem__ treating keys as positions is deprecated. In a future version, integer keys will always be treated as labels (consistent with DataFrame behavior). To access a value by position, use `ser.iloc[pos]`
  center_image_path = data_path + batch_sample[0]


2024-06-21 18-02-29.png    2024-06-23 11-39-10.png
0.0                                          -0.18
Name: 293962, dtype: object
/home/mitnik/Documents/MNA/autonomous_driving/proyecto_final/image_bank/2024-06-23 11-39-10.png


error: OpenCV(4.10.0) /io/opencv/modules/imgproc/src/resize.cpp:4152: error: (-215:Assertion failed) !ssize.empty() in function 'resize'
